## Plant Pathology - The Power of Deep Learning & Computer Vision - Research

<center><img src="https://cals.ncsu.edu/entomology-and-plant-pathology/wp-content/uploads/sites/8/2019/03/scouting-for-disease-950x535.jpg" width="500px"></center>

<font size=3>This is a Plant Pathology Research, I will like to get started on the power of Deep Learning and Computer Vision to diagnose plant diseases by studying plant leaf images. Some categories include "healthy", "rust" and "multiple pathologies".

<font size=3>Learning how to solve this problem, will help understand and get introduce on this field that can be transferred to other research to help having a healthier and brighter future not only for the planet, but for our humanity.

<font size=3>I am looking forward to visualize the data using Seaborn and some Plotly alongside explain as I learn some image processing techniques provided by OpenCV.

<font size=3>Finally, I will show how different pre-trained Tensor Flow models and apply convolution formulas to help find the source of the problem and shed some light for this and oncoming researches.

### Why using TensorFlow? 
<font size=3> As a way to introduce what it can do and why we are going to use it and to learn how it can help further this research ahead. 

<font size=3> Here a video on Why TensorFlow:

In [ ]:
from IPython.display import HTML
HTML('<center><iframe width="560" height="315" src="https://www.youtube.com/embed/yjprpOoH5c8?controls=0" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></center>')

## Table of Contents

# Table of Contents <a id="0"></a>

* [<font size=4>Quick Exploratory Analysis</font>](#1)
    * [1.1. Some techology to use - EfficientNet](#1.1)
    * [1.2. Setting the ground work](#1.2)
    * [1.3. Sample image visualization](#1.3)
    * [1.4. Color Channel distributions and mean values](#1.4)
    * [1.5. Visualize more sample leaves](#1.5)
    * [1.6. Target - Is it Healthy?](#1.6)
    
    
* [<font size=4>Image Processing</font>](#2)
    * [2.1. OpenCV to show images](#2.1)
    * [2.2. Convolution on RGB images](#2.1)
    

* [<font size=4>Models</font>](#3)
* [<font size=4>Conclusions</font>](#4)

* [<font size=4>Thank you note</font>](#5)

* [<font size=4>References</font>](#6)

# Quick Exploratory Analysis <a id="1"></a> - <a id="1"></a> [<font size=4>^</font>](#0)

<font size=3> Some of the new technologies that we will use are called EfficientNet

## Waht is EfficientNet and why use it? <a id="1.1"></a>

<font size=3> EfficientNet is another popular (more recent) Convolutional Neural Network or CNN-based ImageNet model which achieved the State of the Art on several image-based tasks in 2019. EfficientNet performs model scaling in an innovative way to achieve excellent accuracy with significantly fewer parameters. It achieves the same if not greater accuracy than ResNet and DenseNet with a mcuh shallower architecture. Now let us train EfficientNet on leaf images and evaluate its performance.

In [ ]:
!pip install -q efficientnet
print('efficientnet installed')

Setting the ground work

In [ ]:
import os
import gc
import re

import cv2
import math
import numpy as np
import scipy as sp
import pandas as pd

import tensorflow as tf
from IPython.display import SVG
import efficientnet.tfkeras as efn
from keras.utils import plot_model
import tensorflow.keras.layers as L
from keras.utils import model_to_dot
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.applications import DenseNet121

import seaborn as sns
from tqdm import tqdm
import matplotlib.cm as cm
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tqdm.pandas()
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

np.random.seed(0)
tf.random.set_seed(0)

import warnings
warnings.filterwarnings("ignore")

print('Ground work set: (pandas, ploty, numpy, seaborn, sklearn, matplotlib, pyplot, cv2, math, tensor, scipy, IPython, keras and Efficientnet imported)')

Train and Test data 

In [ ]:
img_path = "../input/plant-pathology-2020-fgvc7/images/"
test_path = "../input/plant-pathology-2020-fgvc7/test.csv"
train_path = "../input/plant-pathology-2020-fgvc7/train.csv"
sub_path = "../input/plant-pathology-2020-fgvc7/sample_submission.csv"

In [ ]:
from os import listdir
rl = listdir(img_path)
img = cv2.imread(img_path+rl[0])


b,g,r = cv2.split(img)
print(b,g,r)

In [ ]:
epochs = 30
sample = 50
print(epochs,'epochs and a sample of',sample, 'selected')

### Epoch
<font size=3> What Is an Epoch? The number of epochs is a hyperparameter that defines the number times that the learning algorithm will work through the entire training dataset. One epoch means that each sample in the training dataset has had an opportunity to update the internal model parameters.

In [ ]:
sub = pd.read_csv(sub_path)
test_data = pd.read_csv(test_path)
train_data = pd.read_csv(train_path)
print('sub, test_data and train_data dataframes created')

In [ ]:
print('Train data head')
train_data.head()


In [ ]:
print('Test data head')
test_data.head()

<font size=3> Ok now let's see one of those loaded images on the train set. 

In [ ]:
def image_loading(image_id):
    file_path = image_id + ".jpg"
    image = cv2.imread(img_path + file_path)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

train_images = train_data["image_id"][:sample].progress_apply(image_loading)

<font size=3> images_loading function created and applied to train_images

## **Sample image visualization** <a id="1.2"></a>


<font size=3> Out of the 50 sample here we are selecting the last image, which look in very bad rusting and scab stage. It seems it may have multiple diseases. 

<font size=3> As we we progress on our research we can spot which category is this leave on. </font>

In [ ]:
fig = px.imshow(cv2.resize(train_images[49], (205, 136)))
fig.show()

<font size=3> As you hover over the image you can realize you get the pixel coordinates and a array with the three colors density for red, green and blue or RGB. We are not going to get into much deep explaining the colors, but that is the cuprit for the machine learning models that we will use to spot the images that apper healty from the unhealthy ones.


## Color Channel distributions and mean values <a id ="1.4"></a>

In [ ]:
red_values = [np.mean(train_images[idx][0:, 0:, 0]) for idx in range(len(train_images))]
green_values = [np.mean(train_images[idx][0:, 0:, 1]) for idx in range(len(train_images))]
blue_values = [np.mean(train_images[idx][0:, 0:, 2]) for idx in range(len(train_images))]
values = [np.mean(train_images[idx]) for idx in range(len(train_images))]

In [ ]:
img = cv2.imread(img_path+rl[0])

b,g,r = cv2.split(img)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img)
plt.imshow(r)
plt.imshow(g)
plt.imshow(b)

In [ ]:
fig = go.Figure()

for idx, values in enumerate([red_values, green_values, blue_values]):
    if idx == 0:
        color = "Red"
    if idx == 1:
        color = "Green"
    if idx == 2:
        color = "Blue"
    fig.add_trace(go.Box(x=[color]*len(values), y=values, name=color, marker=dict(color=color.lower())))
    
fig.update_layout(yaxis_title="Mean values", xaxis_title="Color channels",
                  title="Mean values over Color channels", template="plotly_white")

### Color definitions to get the mean values
<font size=3> Red_values, Green_values, Blue_values are each assigned the numpy mean to the index that correspond to the color. 

                               [red, green, blue]
<font size=3> First value in the color array correspone to red, so we skip to index 0, then 1 for green and 2 for blue. 

e.g.:

       red_values = np.mean(train_images[idx][:, :, 0]) for idx in range(len(train_images))

       green_values = np.mean(train_images[idx][:, :, 1]) for idx in range(len(train_images))


## Visualize more sample leaves <a id="1.5"></a>

<font size=3> Now, let us visualize more sample leaves beloning to different categories in the training set.

In [ ]:


import plotly.express as px
import numpy as np
#from skimage import data, filters, img_as_float
img = np.array([r,g,b]) #data.camera()
sigmas = [1, 2, 4]
#img_sequence = [filters.gaussian(img, sigma=sigma) for sigma in sigmas]
fig = px.imshow(img, facet_col=0, binary_string=False,
                labels={'facet_col':'sigma'})
# Set facet titles
for i, sigma in enumerate(sigmas):
    fig.layout.annotations[i]['text'] = 'sigma = %d' %sigma
fig.show()


In [ ]:
import plotly.express as px
import numpy as np

img_rgb = np.array([r,g,b], dtype=np.uint8)
fig = px.imshow(img_rgb, facet_col=0)
fig.show()

In [ ]:
import plotly.graph_objects as go
img_rgb = [[r], [g, [b]]
fig = go.Figure(go.Image(z=img_rgb))
fig.show()

In [ ]:
def visualize_leaves(cond=[0, 0, 0, 0], cond_cols=["healthy"], is_cond=True):
    if not is_cond:
        cols, rows = 3, min([3, len(train_images)//3])
        fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(30, rows*20/3))
        for col in range(cols):
            for row in range(rows):
                ax[row, col].imshow(train_images.loc[train_images.index[-row*3-col-1]])
        return None
        
    cond_0 = "healthy == {}".format(cond[0])
    cond_1 = "scab == {}".format(cond[1])
    cond_2 = "rust == {}".format(cond[2])
    cond_3 = "multiple_diseases == {}".format(cond[3])
    
    cond_list = []
    for col in cond_cols:
        if col == "healthy":
            cond_list.append(cond_0)
        if col == "scab":
            cond_list.append(cond_1)
        if col == "rust":
            cond_list.append(cond_2)
        if col == "multiple_diseases":
            cond_list.append(cond_3)
    
    data = train_data.loc[0:25]
    for cond in cond_list:
        data = data.query(cond)
        
    images = train_images.loc[list(data.index)]
    cols, rows = 3, min([3, len(images)//3])
    
    fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(30, rows*20/3))
    for col in range(cols):
        for row in range(rows):
            ax[row, col].imshow(images.loc[images.index[row*3+col]])
    plt.show()

## Healthy Leaves?

In [ ]:
visualize_leaves(cond=[1, 0, 0, 0], cond_cols=["healthy"])

<font size=3> We can see that the healthy leaves are completely green, do not have any brown/yellow spots or scars. Healthy leaves do not have scab or rust.

## Rusted?

<font size=3> Yes rusted, rust have several brownish-yellow spots across the leaf.

In [ ]:
visualize_leaves(cond=[0, 0, 1, 0], cond_cols=["rust"])

<font size=3> Rust is defined as "a disease, especially of cereals and other grasses, characterized by rust-colored pustules of spores on the affected leaf blades and sheaths and caused by any of several rust fungi". The yellow spots are a sign of infection by a special type of fungi called "rust fungi". Rust can also be treated with several chemical and non-chemical methods once diagnosed.

## Pie Chart

In [ ]:
fig = go.Figure([go.Pie(labels=train_data.columns[1:],
           values=train_data.iloc[:, 1:].sum().values)])
fig.update_layout(title_text="Pie chart of all categories ", template="simple_white")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig.show()

<font size=3> Here we visualize the labels and target data. In all the above chart, green represents the "desired" or "healthy" condition, whearas the others represent the "undesired" or "unhealthy" conditions. Likewise orange for the sab, dark blue for runs and brown for multimple diseases

## Scab ?

<font size=3> Scab have large brown marks and stains across the leaf. 

In [ ]:
visualize_leaves(cond=[0, 1, 0, 0], cond_cols=["scab"])

<font size=3> Scab is defined as "any of various plant diseases caused by fungi or bacteria and resulting in crustlike spots on fruit, leaves, or roots. The spots caused by such a disease". The brown marks across the leaf are a sign of these bacterial/fungal infections. 

Once diagnosed, scab can be treated using chemical or non-chemical methods.

## The target. Is it Healthy? <a id="1.6"><a/>

<font size=3> It is impossible for a healthy leaf (healthy == 1) to have scab, rust, or multiple diseases. Also, every unhealthy leaf has one of either scab, rust, or multiple diseases.

<font size=3> In the pie chart above, we saw that most leaves in the training set are unhealthy by 71.7%. Only 5% of plants have multiple diseases, and "rust" and "scab" occupy approximately 30% of the pie each. 

So, let's dive in to allow convulotional neural networks to help us find and classify healthy and unlhealty leaves. But before that lets see the healthy and unhealhty distribution. 

## Healthy and Not Healthy Distribution Chart

In [ ]:
train_data["Healthy"] = train_data["healthy"].apply(bool).apply(str)
fig = px.histogram(train_data, x="Healthy", title="Healthy & Uhnealthy Distribution", color="Healthy",\
            color_discrete_map={
                "True": px.colors.qualitative.Plotly[0],
                "False": px.colors.qualitative.Plotly[1]})
fig.update_layout(template="simple_white")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig.data[1].marker.line.color = 'rgb(0, 0, 0)'
fig.data[1].marker.line.width = 0.5
fig

<font size=3> We can see the majority or our plants here are unhealthy (healthy == 0). This means that 72% plants in our study are unhealthy and 28% are healthy plants.

## Image processing - Convolution - <a id="2"></a> [<font size=4>^</font>](#0)

In [ ]:
def edge_and_cut(img):
    emb_img = img.copy()
    edges = cv2.Canny(img, 100, 200)
    edge_coors = []
    for i in range(edges.shape[0]):
        for j in range(edges.shape[1]):
            if edges[i][j] != 0:
                edge_coors.append((i, j))
    
    row_min = edge_coors[np.argsort([coor[0] for coor in edge_coors])[0]][0]
    row_max = edge_coors[np.argsort([coor[0] for coor in edge_coors])[-1]][0]
    col_min = edge_coors[np.argsort([coor[1] for coor in edge_coors])[0]][1]
    col_max = edge_coors[np.argsort([coor[1] for coor in edge_coors])[-1]][1]
    new_img = img[row_min:row_max, col_min:col_max]
    
    emb_img[row_min-10:row_min+10, col_min:col_max] = [255, 0, 0]
    emb_img[row_max-10:row_max+10, col_min:col_max] = [255, 0, 0]
    emb_img[row_min:row_max, col_min-10:col_min+10] = [255, 0, 0]
    emb_img[row_min:row_max, col_max-10:col_max+10] = [255, 0, 0]
    
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30, 20))
    ax[0].imshow(img, cmap='gray')
    ax[0].set_title('Original Image', fontsize=24)
    ax[2].imshow(emb_img, cmap='gray')
    ax[2].set_title('Bounding Box', fontsize=24)
    plt.show()

### What is Convolution? <a id="2.2"> </a>
<font size=3> In mathematics (in particular, functional analysis), convolution is a mathematical operation on two functions (f and g) that produces a third function ( ) that expresses how the shape of one is modified by the other. The term convolution refers to both the result function and to the process of computing it.

<center><img src="https://i.ytimg.com/vi/KTB_OFoAQcc/maxresdefault.jpg" width="500px"></center>



In [ ]:
from IPython.display import HTML
HTML('<center><iframe width="560" height="315" src="https://www.youtube.com/embed/KTB_OFoAQcc" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></center>')

## Now Let's convolve some leaves

In [ ]:
def conv(img):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 20))
    kernel = np.ones((7, 7), np.float32)/25
    conv = cv2.filter2D(img, -1, kernel)
    ax[0].imshow(img)
    ax[0].set_title('Original Image', fontsize=24)
    ax[1].imshow(conv)
    ax[1].set_title('Convolved Image', fontsize=24)
    plt.show()

In [ ]:
conv(train_images[3])
conv(train_images[4])
conv(train_images[5])

Convolution is a simple algorithm which involves a kernel (a 2D matrix) which moves over the entire image, calculating dot products with each window along the way. The GIF below demonstrates convolution in action.

## Models <a id="3"></a> - <a id="2"></a> [<font size=4>^</font>](#0)

### EfficientNet 
EfficientNet is another popular (more recent) CNN-based ImageNet model which achieved the SOTA on several image-based tasks in 2019. EfficientNet performs model scaling in an innovative way to achieve excellent accuracy with significantly fewer parameters. It achieves the same if not greater accuracy than ResNet and DenseNet with a mcuh shallower architecture. Now let us train EfficientNet on leaf images and evaluate its performance.

# Tensor Set-UP

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

BATCH_SIZE = 16 * strategy.num_replicas_in_sync # Batch_Size is for memoery and goes in ranges power of two. eg. 16,32, 64,128..
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

# Assigning labesl and defining path

In [ ]:
def format_path(st):
    return GCS_DS_PATH + '/images/' + st + '.jpg'

test_paths = test_data.image_id.apply(format_path).values
train_paths = train_data.image_id.apply(format_path).values

train_labels = np.float32(train_data.loc[:, 'healthy':'scab'].values)
train_paths, valid_paths, train_labels, valid_labels =\
train_test_split(train_paths, train_labels, test_size=0.15, random_state=2020)

# **Strategy Scope Explanation**

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([efn.EfficientNetB7(input_shape=(512, 512, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                 L.GlobalAveragePooling2D(),
                                 L.Dense(train_labels.shape[1],
                                         activation='softmax')])
    
    
        
    model.compile(optimizer='adam',
                  loss = 'categorical_crossentropy',
                  metrics=['categorical_accuracy'])
    model.summary()

Note: each Keras Application expects a specific kind of input preprocessing. For EfficientNet, input preprocessing is included as part of the model (as a Rescaling layer), and thus tf.keras.applications.efficientnet.preprocess_input is actually a pass-through function.

EfficientNet models expect their inputs to be float tensors of pixels with values in the [0-255] range.

### Arguments

#### include_top:
Whether to include the fully-connected layer at the top of the network. Defaults to True.
#### weights: 
One of None (random initialization), 'imagenet' (pre-training on ImageNet), or the path to the weights file to be loaded. Defaults to 'imagenet'.
#### input_tensor: 
Optional Keras tensor (i.e. output of layers.Input()) to use as image input for the model.
input_shape: Optional shape tuple, only to be specified if include_top is False. It should have exactly 3 inputs channels.
#### pooling: 
Optional pooling mode for feature extraction when include_top is False. Defaults to None. - None means that the output of the model will be the 4D tensor output of the last convolutional layer. - avg means that global average pooling will be applied to the output of the last convolutional layer, and thus the output of the model will be a 2D tensor. - max means that global max pooling will be applied.
#### classes: 
Optional number of classes to classify images into, only to be specified if include_top is True, and if no weights argument is specified. Defaults to 1000 (number of ImageNet classes).
classifier_activation: A str or callable. The activation function to use on the "top" layer. Ignored unless include_top=True. Set classifier_activation=None to return the logits of the "top" layer. Defaults to 'softmax'. When loading pretrained weights, classifier_activation can only be None or "softmax".


In [ ]:
def decode_image(filename, label=None, image_size=(512, 512)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if label is None:
        return image
    else:
        return image, label


### Data Set Objects

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_paths, valid_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_paths)
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
)


### Utility functions

In [ ]:
def build_lrfn(lr_start=0.00001, lr_max=0.00005, 
               lr_min=0.00001, lr_rampup_epochs=5, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) *\
                 lr_exp_decay**(epoch - lr_rampup_epochs\
                                - lr_sustain_epochs) + lr_min
        return lr
    return lrfn


### Hyperparameters and Callbacks


In [ ]:
lrfn = build_lrfn()
STEPS_PER_EPOCH = train_labels.shape[0] // BATCH_SIZE
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)

Training the model

In [ ]:
history = model.fit(train_data,
                    epochs=epochs,
                    callbacks=[lr_schedule],
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=valid_dataset)


## Efficientnet predictions
Now, I will visualize some sample predictions made by the EfficientNet model. The red bars represent the model's prediction (maximum probability), the green represent the ground truth (label), and the rest of the bars are blue. When the model predicts correctly, the prediction bar is green.

In [ ]:
def process(img):
    return cv2.resize(img/255.0, (512, 512)).reshape(-1, 512, 512, 3)
def predict(img):
    return model.layers[2](model.layers[1](model.layers[0](process(img)))).numpy()[0]

fig = make_subplots(rows=4, cols=2)
preds = predict(train_images[2])

colors = {"Healthy":px.colors.qualitative.Plotly[0], "Scab":px.colors.qualitative.Plotly[0], "Rust":px.colors.qualitative.Plotly[0], "Multiple diseases":px.colors.qualitative.Plotly[0]}
if list.index(preds.tolist(), max(preds)) == 0:
    pred = "Healthy"
if list.index(preds.tolist(), max(preds)) == 1:
    pred = "Scab"
if list.index(preds.tolist(), max(preds)) == 2:
    pred = "Rust"
if list.index(preds.tolist(), max(preds)) == 3:
    pred = "Multiple diseases"

colors[pred] = px.colors.qualitative.Plotly[1]
colors["Healthy"] = "seagreen"
colors = [colors[val] for val in colors.keys()]
fig.add_trace(go.Image(z=cv2.resize(train_images[2], (205, 136))), row=1, col=1)
fig.add_trace(go.Bar(x=["Healthy", "Multiple diseases", "Rust", "Scab"], y=preds, marker=dict(color=colors)), row=1, col=2)
fig.update_layout(height=1200, width=800, title_text="EfficientNet Predictions", showlegend=False)

preds = predict(train_images[0])
colors = {"Healthy":px.colors.qualitative.Plotly[0], "Scab":px.colors.qualitative.Plotly[0], "Rust":px.colors.qualitative.Plotly[0], "Multiple diseases":px.colors.qualitative.Plotly[0]}
if list.index(preds.tolist(), max(preds)) == 0:
    pred = "Healthy"
if list.index(preds.tolist(), max(preds)) == 1:
    pred = "Multiple diseases"
if list.index(preds.tolist(), max(preds)) == 2:
    pred = "Rust"
if list.index(preds.tolist(), max(preds)) == 3:
    pred = "Scab"
    
colors[pred] = px.colors.qualitative.Plotly[1]
colors["Multiple diseases"] = "seagreen"
colors = [colors[val] for val in colors.keys()]
fig.add_trace(go.Image(z=cv2.resize(train_images[0], (205, 136))), row=2, col=1)
fig.add_trace(go.Bar(x=["Healthy", "Multiple diseases", "Rust", "Scab"], y=preds, marker=dict(color=colors)), row=2, col=2)

preds = predict(train_images[3])
colors = {"Healthy":px.colors.qualitative.Plotly[0], "Scab":px.colors.qualitative.Plotly[0], "Rust":px.colors.qualitative.Plotly[0], "Multiple diseases":px.colors.qualitative.Plotly[0]}
if list.index(preds.tolist(), max(preds)) == 0:
    pred = "Healthy"
if list.index(preds.tolist(), max(preds)) == 1:
    pred = "Multiple diseases"
if list.index(preds.tolist(), max(preds)) == 2:
    pred = "Rust"
if list.index(preds.tolist(), max(preds)) == 3:
    pred = "Scab"
    
colors[pred] = px.colors.qualitative.Plotly[1]
colors["Rust"] = "seagreen"
colors = [colors[val] for val in colors.keys()]
fig.add_trace(go.Image(z=cv2.resize(train_images[3], (205, 136))), row=3, col=1)
fig.add_trace(go.Bar(x=["Healthy", "Multiple diseases", "Rust", "Scab"], y=preds, marker=dict(color=colors)), row=3, col=2)

preds = predict(train_images[1])
colors = {"Healthy":px.colors.qualitative.Plotly[0], "Scab":px.colors.qualitative.Plotly[0], "Rust":px.colors.qualitative.Plotly[0], "Multiple diseases":px.colors.qualitative.Plotly[0]}
if list.index(preds.tolist(), max(preds)) == 0:
    pred = "Healthy"
if list.index(preds.tolist(), max(preds)) == 1:
    pred = "Multiple diseases"
if list.index(preds.tolist(), max(preds)) == 2:
    pred = "Rust"
if list.index(preds.tolist(), max(preds)) == 3:
    pred = "Scab"
    
colors[pred] = px.colors.qualitative.Plotly[1]
colors["Scab"] = "seagreen"
colors = [colors[val] for val in colors.keys()]
fig.add_trace(go.Image(z=cv2.resize(train_images[1], (205, 136))), row=4, col=1)
fig.add_trace(go.Bar(x=["Healthy", "Multiple diseases", "Rust", "Scab"], y=preds, marker=dict(color=colors)), row=4, col=2)
fig.update_layout(template="plotly_white")

<font size=3>Models to be used on our R&D to name a few. We plan to use:
    
1.DenseNet
    
2.EfficientNet
    
3.CNNs and Others.

## Conclusions <a id="4"></a> - <a id="2"></a> [<font size=4>^</font>](#0)

<font size=3> This is still a quick research in progress. Stay tune for coming up versions. </font>

# Thank you note <a id="5"></a> - <a id="2"></a> [<font size=4>^</font>](#0)

<font color="green" size=3>Thank you for reading my research and development. Please comment and upvote if you like it to move my learning developent forward. It motivates me to continue my journey to learn more about Data Science for the good of our society :)</font>

## References <a id="6"></a> - <a id="2"></a> [<font size=4>^</font>](#0)

Rectified Linear Activation Unit (RELU) https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/

Convolution - Wikipedia https://en.wikipedia.org/wiki/Convolution#:~:text=In%20mathematics%20

Kernel Image processing - https://en.wikipedia.org/wiki/Kernel_(image_processing)

[<font size=4>Back to the table of contents</font>](#0)